# 1. Environment Set-up

In [1]:
import pyspark
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext 
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import *

groupproject = SparkSession.builder.appName("HM_Recommendations").config("spark.sql.files.maxPartitionBytes", 5000000).getOrCreate()
spark = SparkSession(groupproject)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/25 16:55:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/25 16:55:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# 2. Dataset check & data choose

In [2]:
transaction = spark.read.option("header",True) \
              .csv("transactions_train.csv")
transaction.printSchema()

root
 |-- t_dat: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- article_id: string (nullable = true)
 |-- price: string (nullable = true)
 |-- sales_channel_id: string (nullable = true)



In [6]:
from pyspark.sql.functions import min, max
from pyspark.sql.functions import unix_timestamp, lit
min_date, max_date = transaction.select(min("t_dat"), max("t_dat")).first()
print(min_date, max_date)

datahm =  transaction.withColumn('t_dat', transaction['t_dat'].cast('string'))
datahm = datahm.withColumn('date', from_unixtime(unix_timestamp('t_dat', 'yyyy-MM-dd')))
datahm = datahm.withColumn('year', year(col('date')))
datahm = datahm.withColumn('month', month(col('date')))
datahm = datahm.withColumn('day', date_format(col('date'), "d"))

datahm = datahm[datahm['year'] == 2020]
datahm = datahm[datahm['month'] == 9]
datahm = datahm[datahm['day'] == 22]
transaction.unpersist()

# Prepare the dataset
datahm = datahm.groupby('customer_id', 'article_id').count()
datahm.show()

2018-09-20 2020-09-22


+--------------------+----------+-----+
|         customer_id|article_id|count|
+--------------------+----------+-----+
|00f7bc5c0df4c615b...|0780418013|    1|
|02094817e46f3b692...|0791587001|    1|
|0333e5dda0257e9f4...|0839332002|    2|
|07c7a1172caf8fb97...|0573085043|    1|
|081373184e601470c...|0714790020|    1|
|09bec2a61046ccbea...|0860336002|    1|
|0be4f1ecce204ee32...|0573085028|    1|
|0c4b30343292b5101...|0918522001|    1|
|0e10e02358875468b...|0579541001|    1|
|0fc371e67e61a31d7...|0907170001|    1|
|10817b19177f6a53e...|0718278019|    1|
|10ac90988da6052dd...|0934212003|    1|
|14a298482fa7f9d52...|0894353002|    1|
|14f4b1b17991c32d2...|0803685001|    1|
|1601fa3c3f39aa623...|0730683001|    1|
|164e1a251f0e3d764...|0831267001|    1|
|165d2c0b0128d5619...|0909081004|    1|
|166546028742fe655...|0767423013|    1|
|189b7275c513a84c8...|0877711001|    1|
|1918933afff08e955...|0914672001|    1|
+--------------------+----------+-----+
only showing top 20 rows



In [4]:
userId_count = datahm.groupBy("customer_id").count().orderBy('count', ascending=False)
userId_count.show()

+--------------------+-----+
|         customer_id|count|
+--------------------+-----+
|30b6056bacc5f5c9d...|   28|
|5e8fb4d457fdffc61...|   28|
|dc1b173e541f8d3c1...|   27|
|6335d496ef463bc40...|   25|
|1796e87fd2e88932b...|   25|
|f50287d9cf052d4b4...|   24|
|54e8ebd39543b5a4d...|   23|
|fd5ce8716faf00f6a...|   23|
|850ec77661a417d27...|   22|
|ad3663a848dccbdda...|   21|
|32f3a6a7ce63d302c...|   21|
|b606fe5786c00151a...|   21|
|298523b6637340717...|   21|
|b49647f84a99ced53...|   21|
|fc783381f1ea2174c...|   21|
|a08e284bb18add2d7...|   21|
|383e1b07e2c1fe169...|   21|
|3ca77aab50ae4532b...|   20|
|2a721767cd9864ed5...|   20|
|af5166e0f89b0d433...|   19|
+--------------------+-----+
only showing top 20 rows



In [5]:
articleId_count = datahm.groupBy("article_id").count().orderBy('count', ascending=False)
articleId_count.show()

+----------+-----+
|article_id|count|
+----------+-----+
|0924243002|   91|
|0918522001|   88|
|0866731001|   78|
|0751471001|   75|
|0448509014|   73|
|0714790020|   72|
|0762846027|   68|
|0928206001|   67|
|0893432002|   66|
|0918292001|   65|
|0915529005|   64|
|0788575004|   63|
|0915529003|   63|
|0863583001|   60|
|0930380001|   59|
|0573085028|   59|
|0919273002|   58|
|0850917001|   57|
|0573085042|   56|
|0874110016|   53|
+----------+-----+
only showing top 20 rows



# 3. Model build & Evaluator generate

In [7]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(datahm.columns)-set(['count'])) ]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(datahm).transform(datahm)
transformed.show()

22/10/25 17:02:43 WARN DAGScheduler: Broadcasting large task binary with size 1210.8 KiB
+--------------------+----------+-----+-----------------+----------------+
|         customer_id|article_id|count|customer_id_index|article_id_index|
+--------------------+----------+-----+-----------------+----------------+
|00f7bc5c0df4c615b...|0780418013|    1|            783.0|          2237.0|
|02094817e46f3b692...|0791587001|    1|            785.0|            35.0|
|0333e5dda0257e9f4...|0839332002|    2|           4098.0|           732.0|
|07c7a1172caf8fb97...|0573085043|    1|           1702.0|            44.0|
|081373184e601470c...|0714790020|    1|           4146.0|             5.0|
|09bec2a61046ccbea...|0860336002|    1|           6792.0|          2368.0|
|0be4f1ecce204ee32...|0573085028|    1|            799.0|            14.0|
|0c4b30343292b5101...|0918522001|    1|           6825.0|             1.0|
|0e10e02358875468b...|0579541001|    1|           2689.0|            53.0|
|0fc371e67e

In [8]:
(training,test)=transformed.randomSplit([0.8, 0.2])

In [10]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


#create ALS model
als=ALS(userCol="customer_id_index",itemCol="article_id_index",ratingCol="count",coldStartStrategy="drop",nonnegative=True)

#tune model using ParamGridBuilder
param_grid = ParamGridBuilder()\
            .addGrid(als.rank, [15,20,25])\
            .addGrid(als.maxIter,[5,10,15])\
            .addGrid(als.regParam,[0.05,0.range10,0.15])\
            .build()
#define evaluator as RMSE
evaluator = RegressionEvaluator(metricName = "rmse",labelCol = 'count', predictionCol = 'prediction')

#Build cross validation using CrossValidator
crossvalidate = CrossValidator(estimator=als,estimatorParamMaps=param_grid, evaluator=evaluator,numFolds=3)

#load the crovalidator into the model
model = crossvalidate.fit(training)

22/10/25 17:21:46 WARN DAGScheduler: Broadcasting large task binary with size 1270.7 KiB
22/10/25 17:21:47 WARN DAGScheduler: Broadcasting large task binary with size 1272.7 KiB
22/10/25 17:21:47 WARN DAGScheduler: Broadcasting large task binary with size 1271.3 KiB


22/10/25 17:21:47 WARN DAGScheduler: Broadcasting large task binary with size 1228.5 KiB
22/10/25 17:21:47 WARN DAGScheduler: Broadcasting large task binary with size 1280.3 KiB
22/10/25 17:21:47 WARN DAGScheduler: Broadcasting large task binary with size 1278.9 KiB


22/10/25 17:22:27 WARN DAGScheduler: Broadcasting large task binary with size 1341.9 KiB
22/10/25 17:22:27 WARN DAGScheduler: Broadcasting large task binary with size 1237.4 KiB
22/10/25 17:22:27 WARN DAGScheduler: Broadcasting large task binary with size 1237.4 KiB
22/10/25 17:22:27 WARN DAGScheduler: Broadcasting large task binary with size 1238.9 KiB
22/10/25 17:22:27 WARN DAGScheduler: Broadcasting large task binary with size 1238.8 KiB


22/10/25 17:22:31 WARN DAGScheduler: Broadcasting large task binary with size 1240.4 KiB


22/10/25 17:22:32 WARN DAGScheduler: Broadcasting large task binary with size 1241.7 KiB


...

22/10/25 17:42:22 WARN DAGScheduler: Broadcasting large task binary with size 1240.5 KiB


22/10/25 17:42:24 WARN DAGScheduler: Broadcasting large task binary with size 1241.8 KiB
22/10/25 17:42:24 WARN DAGScheduler: Broadcasting large task binary with size 1240.8 KiB


22/10/25 17:42:25 WARN DAGScheduler: Broadcasting large task binary with size 1242.1 KiB
22/10/25 17:42:25 WARN DAGScheduler: Broadcasting large task binary with size 1242.8 KiB
22/10/25 17:42:25 WARN DAGScheduler: Broadcasting large task binary with size 1245.9 KiB
22/10/25 17:42:26 WARN DAGScheduler: Broadcasting large task binary with size 1247.3 KiB
22/10/25 17:42:26 WARN DAGScheduler: Broadcasting large task binary with size 1248.7 KiB
22/10/25 17:42:26 WARN DAGScheduler: Broadcasting large task binary with size 1250.1 KiB
22/10/25 17:42:26 WARN DAGScheduler: Broadcasting large task binary with size 1251.5 KiB
22/10/25 17:42:26 WARN DAGScheduler: Broadcasting large task binary with size 1252.9 KiB
22/10/25 17:42:26 WARN DAGScheduler: Broadcasting large task binary with size 1254.3 KiB
22/10/25 17:42:26 WARN DAGScheduler: Broadcasting large task binary with size 1255.6 KiB
22/10/25 17:42:26 WARN DAGScheduler: Broadcasting large task binary with size 1257.0 KiB
22/10/25 17:42:26 WAR

22/10/25 17:42:32 WARN DAGScheduler: Broadcasting large task binary with size 1355.6 KiB


22/10/25 17:43:51 WARN DAGScheduler: Broadcasting large task binary with size 1227.3 KiB
22/10/25 17:43:51 WARN DAGScheduler: Broadcasting large task binary with size 1228.7 KiB
22/10/25 17:43:51 WARN DAGScheduler: Broadcasting large task binary with size 1230.2 KiB
22/10/25 17:43:51 WARN DAGScheduler: Broadcasting large task binary with size 1231.5 KiB
22/10/25 17:43:52 WARN DAGScheduler: Broadcasting large task binary with size 1230.5 KiB
22/10/25 17:43:52 WARN DAGScheduler: Broadcasting large task binary with size 1231.8 KiB
22/10/25 17:43:52 WARN DAGScheduler: Broadcasting large task binary with size 1232.5 KiB
22/10/25 17:43:52 WARN DAGScheduler: Broadcasting large task binary with size 1235.7 KiB
22/10/25 17:43:52 WARN DAGScheduler: Broadcasting large task binary with size 1237.1 KiB
22/10/25 17:43:52 WARN DAGScheduler: Broadcasting large task binary with size 1238.4 KiB
22/10/25 17:43:53 WARN DAGScheduler: Broadcasting large task binary with size 1239.8 KiB
22/10/25 17:43:53 WAR

In [11]:
als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="customer_id_index",itemCol="article_id_index",ratingCol="count",coldStartStrategy="drop",nonnegative=True)
model=als.fit(training)

22/10/25 17:45:52 WARN DAGScheduler: Broadcasting large task binary with size 1227.3 KiB
22/10/25 17:45:52 WARN DAGScheduler: Broadcasting large task binary with size 1228.7 KiB


22/10/25 17:45:52 WARN DAGScheduler: Broadcasting large task binary with size 1230.2 KiB
22/10/25 17:45:53 WARN DAGScheduler: Broadcasting large task binary with size 1231.5 KiB
22/10/25 17:45:53 WARN DAGScheduler: Broadcasting large task binary with size 1230.5 KiB
22/10/25 17:45:53 WARN DAGScheduler: Broadcasting large task binary with size 1231.8 KiB
22/10/25 17:45:53 WARN DAGScheduler: Broadcasting large task binary with size 1232.5 KiB
22/10/25 17:45:53 WARN DAGScheduler: Broadcasting large task binary with size 1235.7 KiB
22/10/25 17:45:53 WARN DAGScheduler: Broadcasting large task binary with size 1237.1 KiB
22/10/25 17:45:53 WARN DAGScheduler: Broadcasting large task binary with size 1238.4 KiB
22/10/25 17:45:53 WARN DAGScheduler: Broadcasting large task binary with size 1239.8 KiB
22/10/25 17:45:53 WARN DAGScheduler: Broadcasting large task binary with size 1241.2 KiB
22/10/25 17:45:54 WARN DAGScheduler: Broadcasting large task binary with size 1242.6 KiB
22/10/25 17:45:54 WAR

# 4.Prediction & Evaluate


In [12]:
predictions = model.transform(test)
rmse = evaluator.evaluate(predictions)

22/10/25 17:49:34 WARN DAGScheduler: Broadcasting large task binary with size 1256.4 KiB
22/10/25 17:49:34 WARN DAGScheduler: Broadcasting large task binary with size 1255.0 KiB


22/10/25 17:50:19 WARN DAGScheduler: Broadcasting large task binary with size 1288.1 KiB
22/10/25 17:50:20 WARN DAGScheduler: Broadcasting large task binary with size 1300.0 KiB


In [13]:
#print evaluation metrics and model parameters
print("RMSE =" + str(rmse))

RMSE =0.47325511969985745


In [31]:
##find the best model
rank_1 = []
maxIter_1 = []
regParam_1 = []
rmse_1 = []
regParam = [0.01, 0.02,0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]
for i in range(5,25,5):
    for n in range(5,25,5):
        for m in regParam:
            als=ALS(maxIter=n,regParam=m,rank=i,userCol="customer_id_index",itemCol="article_id_index",ratingCol="count",coldStartStrategy="drop",nonnegative=True)
            model=als.fit(training)
            predictions = model.transform(test)
            rmse = evaluator.evaluate(predictions)
            rmse = str(rmse)
            rank_1.append(i)
            maxIter_1.append(n)
            regParam_1.append(m)
            rmse_1.append(rmse)

22/10/25 20:48:53 WARN DAGScheduler: Broadcasting large task binary with size 1227.3 KiB
22/10/25 20:48:53 WARN DAGScheduler: Broadcasting large task binary with size 1228.7 KiB
22/10/25 20:48:53 WARN DAGScheduler: Broadcasting large task binary with size 1230.2 KiB
22/10/25 20:48:53 WARN DAGScheduler: Broadcasting large task binary with size 1231.5 KiB
22/10/25 20:48:53 WARN DAGScheduler: Broadcasting large task binary with size 1230.5 KiB
22/10/25 20:48:53 WARN DAGScheduler: Broadcasting large task binary with size 1231.8 KiB
22/10/25 20:48:53 WARN DAGScheduler: Broadcasting large task binary with size 1232.5 KiB
22/10/25 20:48:53 WARN DAGScheduler: Broadcasting large task binary with size 1235.7 KiB
22/10/25 20:48:54 WARN DAGScheduler: Broadcasting large task binary with size 1237.1 KiB
22/10/25 20:48:54 WARN DAGScheduler: Broadcasting large task binary with size 1238.4 KiB
22/10/25 20:48:54 WARN DAGScheduler: Broadcasting large task binary with size 1239.8 KiB
22/10/25 20:48:54 WAR

22/10/25 20:49:37 WARN DAGScheduler: Broadcasting large task binary with size 1288.1 KiB
22/10/25 20:49:38 WARN DAGScheduler: Broadcasting large task binary with size 1300.0 KiB


22/10/25 20:50:20 WARN DAGScheduler: Broadcasting large task binary with size 1227.3 KiB
22/10/25 20:50:20 WARN DAGScheduler: Broadcasting large task binary with size 1228.7 KiB
22/10/25 20:50:20 WARN DAGScheduler: Broadcasting large task binary with size 1230.2 KiB
22/10/25 20:50:20 WARN DAGScheduler: Broadcasting large task binary with size 1231.5 KiB
22/10/25 20:50:20 WARN DAGScheduler: Broadcasting large task binary with size 1230.5 KiB
22/10/25 20:50:20 WARN DAGScheduler: Broadcasting large task binary with size 1231.8 KiB
22/10/25 20:50:21 WARN DAGScheduler: Broadcasting large task binary with size 1232.5 KiB
22/10/25 20:50:21 WARN DAGScheduler: Broadcasting large task binary with size 1235.7 KiB
22/10/25 20:50:21 WARN DAGScheduler: Broadcasting large task binary with size 1237.1 KiB
22/10/25 20:50:21 WARN DAGScheduler: Broadcasting large task binary with size 1238.4 KiB
22/10/25 20:50:21 WARN DAGScheduler: Broadcasting large task binary with size 1239.8 KiB
22/10/25 20:50:21 WAR

22/10/25 20:51:16 WARN DAGScheduler: Broadcasting large task binary with size 1288.1 KiB
22/10/25 20:51:16 WARN DAGScheduler: Broadcasting large task binary with size 1300.0 KiB


22/10/25 20:52:05 WARN DAGScheduler: Broadcasting large task binary with size 1227.3 KiB
22/10/25 20:52:05 WARN DAGScheduler: Broadcasting large task binary with size 1228.7 KiB
22/10/25 20:52:05 WARN DAGScheduler: Broadcasting large task binary with size 1230.2 KiB
22/10/25 20:52:05 WARN DAGScheduler: Broadcasting large task binary with size 1231.5 KiB
22/10/25 20:52:05 WARN DAGScheduler: Broadcasting large task binary with size 1230.5 KiB
22/10/25 20:52:05 WARN DAGScheduler: Broadcasting large task binary with size 1231.8 KiB
22/10/25 20:52:06 WARN DAGScheduler: Broadcasting large task binary with size 1232.5 KiB
22/10/25 20:52:06 WARN DAGScheduler: Broadcasting large task binary with size 1235.7 KiB
22/10/25 20:52:06 WARN DAGScheduler: Broadcasting large task binary with size 1237.1 KiB
22/10/25 20:52:06 WARN DAGScheduler: Broadcasting large task binary with size 1238.4 KiB
22/10/25 20:52:06 WARN DAGScheduler: Broadcasting large task binary with size 1239.8 KiB
22/10/25 20:52:06 WAR

...

22/10/26 00:38:13 WARN DAGScheduler: Broadcasting large task binary with size 1227.3 KiB
22/10/26 00:38:13 WARN DAGScheduler: Broadcasting large task binary with size 1228.7 KiB
22/10/26 00:38:13 WARN DAGScheduler: Broadcasting large task binary with size 1230.2 KiB
22/10/26 00:38:13 WARN DAGScheduler: Broadcasting large task binary with size 1231.5 KiB
22/10/26 00:38:13 WARN DAGScheduler: Broadcasting large task binary with size 1230.5 KiB
22/10/26 00:38:13 WARN DAGScheduler: Broadcasting large task binary with size 1231.8 KiB
22/10/26 00:38:14 WARN DAGScheduler: Broadcasting large task binary with size 1232.5 KiB
22/10/26 00:38:14 WARN DAGScheduler: Broadcasting large task binary with size 1235.7 KiB
22/10/26 00:38:14 WARN DAGScheduler: Broadcasting large task binary with size 1237.1 KiB
22/10/26 00:38:14 WARN DAGScheduler: Broadcasting large task binary with size 1238.4 KiB
22/10/26 00:38:14 WARN DAGScheduler: Broadcasting large task binary with size 1239.8 KiB
22/10/26 00:38:14 WAR

22/10/26 00:38:48 WARN DAGScheduler: Broadcasting large task binary with size 1329.7 KiB
22/10/26 00:38:48 WARN DAGScheduler: Broadcasting large task binary with size 1341.6 KiB


22/10/26 00:39:18 WARN DAGScheduler: Broadcasting large task binary with size 1227.3 KiB
22/10/26 00:39:18 WARN DAGScheduler: Broadcasting large task binary with size 1228.7 KiB
22/10/26 00:39:18 WARN DAGScheduler: Broadcasting large task binary with size 1230.2 KiB


22/10/26 00:39:18 WARN DAGScheduler: Broadcasting large task binary with size 1231.5 KiB
22/10/26 00:39:18 WARN DAGScheduler: Broadcasting large task binary with size 1230.5 KiB
22/10/26 00:39:18 WARN DAGScheduler: Broadcasting large task binary with size 1231.8 KiB
22/10/26 00:39:19 WARN DAGScheduler: Broadcasting large task binary with size 1232.5 KiB
22/10/26 00:39:19 WARN DAGScheduler: Broadcasting large task binary with size 1235.7 KiB
22/10/26 00:39:19 WARN DAGScheduler: Broadcasting large task binary with size 1237.1 KiB
22/10/26 00:39:19 WARN DAGScheduler: Broadcasting large task binary with size 1238.4 KiB
22/10/26 00:39:19 WARN DAGScheduler: Broadcasting large task binary with size 1239.8 KiB
22/10/26 00:39:20 WARN DAGScheduler: Broadcasting large task binary with size 1241.2 KiB
22/10/26 00:39:20 WARN DAGScheduler: Broadcasting large task binary with size 1242.6 KiB
22/10/26 00:39:20 WARN DAGScheduler: Broadcasting large task binary with size 1244.0 KiB
22/10/26 00:39:20 WAR

22/10/26 00:39:53 WARN DAGScheduler: Broadcasting large task binary with size 1329.7 KiB
22/10/26 00:39:53 WARN DAGScheduler: Broadcasting large task binary with size 1341.6 KiB


In [34]:
df_1 = {"rank":rank_1, "maxIter":maxIter_1,  "regParam": regParam_1, "RMSE": rmse_1}
df_1 = pd.DataFrame(df_1)
df_1.sort_values(by='RMSE')

,rank,maxIter,regParam,RMSE
154,20,20,0.05,0.43621867505918144
155,20,20,0.06,0.43628881937955905
117,15,20,0.08,0.4363435207448502
118,15,20,0.09,0.43645635666758403
156,20,20,0.07,0.4365432614916436
...,...,...,...,...
10,5,10,0.01,0.5894333765079824
80,15,5,0.01,0.5905611623791649
1,5,5,0.02,0.5974632794488814
40,10,5,0.01,0.6237407257601445


In [64]:
## run the best model
als=ALS(maxIter=20,regParam=0.05,rank=20,userCol="customer_id_index",itemCol="article_id_index",ratingCol="count",coldStartStrategy="drop",nonnegative=True)
bestmodel=als.fit(training)
predictions = bestmodel.transform(test)

22/10/26 09:54:58 WARN DAGScheduler: Broadcasting large task binary with size 1227.3 KiB
22/10/26 09:54:58 WARN DAGScheduler: Broadcasting large task binary with size 1228.7 KiB
22/10/26 09:54:58 WARN DAGScheduler: Broadcasting large task binary with size 1230.2 KiB


22/10/26 09:54:58 WARN DAGScheduler: Broadcasting large task binary with size 1231.5 KiB
22/10/26 09:54:58 WARN DAGScheduler: Broadcasting large task binary with size 1230.5 KiB
22/10/26 09:54:58 WARN DAGScheduler: Broadcasting large task binary with size 1231.8 KiB
22/10/26 09:54:59 WARN DAGScheduler: Broadcasting large task binary with size 1232.5 KiB
22/10/26 09:54:59 WARN DAGScheduler: Broadcasting large task binary with size 1235.7 KiB
22/10/26 09:54:59 WARN DAGScheduler: Broadcasting large task binary with size 1237.1 KiB
22/10/26 09:54:59 WARN DAGScheduler: Broadcasting large task binary with size 1238.4 KiB
22/10/26 09:54:59 WARN DAGScheduler: Broadcasting large task binary with size 1239.8 KiB
22/10/26 09:54:59 WARN DAGScheduler: Broadcasting large task binary with size 1241.2 KiB
22/10/26 09:54:59 WARN DAGScheduler: Broadcasting large task binary with size 1242.6 KiB
22/10/26 09:54:59 WARN DAGScheduler: Broadcasting large task binary with size 1244.0 KiB
22/10/26 09:55:00 WAR

In [87]:
recommendation_customer = bestmodel.recommendForAllUsers(20)
recommendation_customer.show(20)

22/10/26 10:07:56 WARN DAGScheduler: Broadcasting large task binary with size 1343.7 KiB


22/10/26 10:08:02 WARN DAGScheduler: Broadcasting large task binary with size 1321.6 KiB
+-----------------+--------------------+
|customer_id_index|     recommendations|
+-----------------+--------------------+
|               26|[{1661, 2.5188386...|
|               27|[{1661, 2.3672147...|
|               28|[{1661, 2.366427}...|
|               31|[{5111, 2.1222975...|
|               34|[{1661, 2.4375648...|
|               44|[{5111, 2.482372}...|
|               53|[{1661, 3.0103657...|
|               65|[{5111, 3.5202312...|
|               76|[{1661, 2.3918033...|
|               78|[{1661, 2.4072773...|
|               81|[{1661, 3.4048162...|
|               85|[{1661, 2.28756},...|
|              101|[{4249, 2.9081094...|
|              103|[{5111, 3.6313312...|
|              108|[{1661, 2.2926984...|
|              115|[{5040, 2.6389184...|
|              126|[{1661, 2.4568224...|
|              133|[{5111, 2.3707392...|
|              137|[{6383, 3.1098046...|
|        

# 5 Providing Recommendations

In [88]:
recommendation_customer = recommendation_customer.select("customer_id_index","recommendations.article_id_index")
recommendation_customer.show(20)
recommendation_customer = recommendation_customer.toPandas()

22/10/26 10:08:02 WARN DAGScheduler: Broadcasting large task binary with size 1343.7 KiB


22/10/26 10:08:07 WARN DAGScheduler: Broadcasting large task binary with size 1321.8 KiB
+-----------------+--------------------+
|customer_id_index|    article_id_index|
+-----------------+--------------------+
|               26|[1661, 5040, 6383...|
|               27|[1661, 5111, 4405...|
|               28|[1661, 5111, 4405...|
|               31|[5111, 1661, 4405...|
|               34|[1661, 5111, 4405...|
|               44|[5111, 1661, 3018...|
|               53|[1661, 5111, 6383...|
|               65|[5111, 1661, 4405...|
|               76|[1661, 5111, 4405...|
|               78|[1661, 5111, 5040...|
|               81|[1661, 5040, 6383...|
|               85|[1661, 5111, 4405...|
|              101|[4249, 4212, 5111...|
|              103|[5111, 1661, 4405...|
|              108|[1661, 5111, 4405...|
|              115|[5040, 6383, 5111...|
|              126|[1661, 5111, 5040...|
|              133|[5111, 1661, 5040...|
|              137|[6383, 5040, 1661...|
|        

22/10/26 10:08:12 WARN DAGScheduler: Broadcasting large task binary with size 1321.7 KiB


In [90]:
df_trans = transformed.select(transformed['article_id'],transformed['article_id_index']
                        ,transformed['customer_id'],transformed['customer_id_index'])
df_trans = df_trans.toPandas()
df_trans

22/10/26 10:08:41 WARN DAGScheduler: Broadcasting large task binary with size 1207.8 KiB


,article_id,article_id_index,customer_id,customer_id_index
0,0780418013,2237.0,00f7bc5c0df4c615b2502a2c2e9ef9eff988c81dec2e5e...,783.0
1,0791587001,35.0,02094817e46f3b692149b06cf9577e42848c2294e78598...,785.0
2,0839332002,732.0,0333e5dda0257e9f498be52f1e569bfae576caed0cbdcd...,4098.0
3,0573085043,44.0,07c7a1172caf8fb9784b28e51b25b985ab6a1ec7ce923e...,1702.0
4,0714790020,5.0,081373184e601470cc9911f33d3eeebc6f33ed79222573...,4146.0
...,...,...,...,...
29481,0817150004,851.0,f8156f726aeaf44e90c1988837e13c9c9974ee19009e5b...,2594.0
29482,0893432005,96.0,f825202d015506981dc42c53afb7b56a36e05c85b67886...,1657.0
29483,0799754001,5499.0,faed38aaccd80db66f5a1581fe99af84e79fe398c91899...,10465.0
29484,0928206001,7.0,fcffcb9777aab7a53e3b382a840958d800e6d53bdd8a20...,4052.0


In [91]:
dict1 =dict(zip(df_trans['article_id_index'],df_trans['article_id']))
dict2 = dict(zip(df_trans['customer_id_index'],df_trans['customer_id']))
recommendation_customer['article_id'] = recommendation_customer['article_id_index'].map(
    lambda x: [dict1[y] for y in x if y in dict1])
recommendation_customer['customer_id']=recommendation_customer['customer_id_index'].map(dict2)
recommendation_customer.drop(['customer_id_index','article_id_index'], axis = 1)
recommendation = recommendation_customer[['customer_id','article_id']]
recommendation

,customer_id,article_id
0,49725e6a9c754dc6fa7514850e0fc443a2c7d5bf19adac...,"[0297078008, 0750481010, 0857347002, 075797100..."
1,596303c1bf3f84a300a4292424285a392e01c54fdec5f8...,"[0297078008, 0757971006, 0571048002, 085734700..."
2,03c25de221c0d5529471240e2be885ba48065bc6cdee18...,"[0297078008, 0757971006, 0571048002, 085734700..."
3,32210e981f60ef8bc523c4b60dea3a61f2f0bad1746bdc...,"[0757971006, 0297078008, 0571048002, 090892700..."
4,904eae6302d0d36b27e76fb3462b4df85ab61ac4fc3118...,"[0297078008, 0757971006, 0571048002, 075048101..."
...,...,...
9634,fcebefd782268cb541fbfe5d7bb2645463575e54d37c7f...,"[0297078008, 0757971006, 0571048002, 090496100..."
9635,fd42fe1c4a3c3ddebdc413ff70f99d28d218cdc81384cf...,"[0871638002, 0757971006, 0904961003, 075048101..."
9636,fdb8852e111e2dfc5128ecb36506e19538a0c3159451ec...,"[0297078008, 0757971006, 0825109005, 090496100..."
9637,fed6aeb7fabd3ca108f474b7ed3bb80ca33103e9b6a7d6...,"[0297078008, 0757971006, 0571048002, 087163800..."


In [92]:
recommendation.to_csv('recommendation.csv',index=False)